In [ ]:
###### Mount Point 1 through Oauth security.
storageAccount = "gen10dbcdatalake"
storageContainer = "group1-capstone"
clientSecret = 'hidden for security purposes'
clientid = "hidden for security purposes"
mount_point = "/mnt/group1-capstone"
#20200906-20201006/Detroit911-20200906-20201006.csv


configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(
source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
mount_point = mount_point,
extra_configs = configs)

/mnt/group1-capstone has been unmounted.
Out[1]: True

In [ ]:
# Create Error Messages
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [ ]:
# Connect to the topic
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
#confluentTopicName = "capstone-group1"
confluentTopicName = "capstone-group1-topic"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "hidden for security purposes"
confluentSecret = 'hidden for security purposes'
confluentRegistryApiKey = 'hidden for security purposes'
confluentRegistrySecret = 'hidden for security purposes'


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

c.subscribe(['capstone-group1-topic'])

In [ ]:
aString = {}

kafkaListDictionaries = []

while(True):
    try:
        msg = c.poll(timeout=1.0)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            aString['timestamp'] = msg.timestamp()[1]
            kafkaListDictionaries.append(aString)
    except Exception as e:
        print(e)

    for message in kafkaListDictionaries:
        print(message)

{'year': '1991', 'country': 'US', 'Stocks traded, total value (% of GDP)': 34.7482522694799, 'Inflation, consumer prices (annual %)': 4.23496396453853, 'Stocks traded, turnover ratio of domestic shares (%)': 51.4435202733189, 'Tax revenue (% of GDP)': 10.3203424286825, 'GDP per capita (current US$)': 24342.2589048189, 'Central government debt, total (% of GDP)': 44.0615972806026, 'Gini index (World Bank estimate)': 38, 'Broad money (% of GDP)': 70.0032428680854, 'Total reserves (includes gold, current US$)': 159272850288.003, 'timestamp': 1635359356949}
{'year': '1991', 'country': 'US', 'Stocks traded, total value (% of GDP)': 34.7482522694799, 'Inflation, consumer prices (annual %)': 4.23496396453853, 'Stocks traded, turnover ratio of domestic shares (%)': 51.4435202733189, 'Tax revenue (% of GDP)': 10.3203424286825, 'GDP per capita (current US$)': 24342.2589048189, 'Central government debt, total (% of GDP)': 44.0615972806026, 'Gini index (World Bank estimate)': 38, 'Broad money (% of GDP)': 70.0032428680854, 'Total reserves (includes gold, current US$)': 159272850288.003, 'timestamp': 1635359356949}
{'year': '1992', 'country': 'US', 'Stocks traded, total value (% of GDP)': 38.8086241686958, 'Inflation, consumer prices (annual %)': 3.0288196781497, 'Stocks traded, turnover ratio of domestic shares (%)': 55.665113013117, 'Tax revenue (% of GDP)': 9.98416183728209, 'GDP per capita (current US$)': 25418.9907763319, 'Central government debt, total (% of GDP)': 46.0501444176036, 'Gini index (World Bank estimate)': 38.4, 'Broad money (% of GDP)': 66.0133609863432, 'Total reserves (includes gold, current US$)': 147525874735.105, 'timestamp': 1635359362235}
{'year': '1991', 'country': 'US', 'Stocks traded, total value (% of GDP)': 34.7482522694799, 'Inflation, consumer prices (annual %)': 4.23496396453853, 'Stocks traded, turnover ratio of domestic shares (%)': 51.4435202733189, 'Tax revenue (% of GDP)': 10.3203424286825, 'GDP per capita (current US$)': 24342.2589048189, 'Central government debt, total (% of GDP)': 44.0615972806026, 'Gini index (World Bank estimate)': 38, 'Broad money (% of GDP)': 70.0032428680854, 'Total reserves (includes gold, current US$)': 159272850288.003, 'timestamp': 1635359356949}
{'year': '1992', 'country': 'US', 'Stocks traded, total value (% of GDP)': 38.8086241686958, 'Inflation, consumer prices (annual %)': 3.0288196781497, 'Stocks traded, turnover ratio of domestic shares (%)': 55.665113013117, 'Tax revenue (% of GDP)': 9.98416183728209, 'GDP per capita (current US$)': 25418.9907763319, 'Central government debt, total (% of GDP)': 46.0501444176036, 'Gini index (World Bank estimate)': 38.4, 'Broad money (% of GDP)': 66.0133609863432, 'Total reserves (includes gold, current US$)': 147525874735.105, 'timestamp': 1635359362235}
{'year': '1993', 'country': 'US', 'Stocks traded, total value (% of GDP)': 50.0748378777525, 'Inflation, consumer prices (annual %)': 2.95165696638554, 'Stocks traded, turnover ratio of domestic shares (%)': 65.4039272301037, 'Tax revenue (% of GDP)': 10.3049343163775, 'GDP per capita (current US$)': 26387.2937338171, 'Central government debt, total (% of GDP)': 48.2461403335599, 'Gini index (World Bank estimate)': 40.4, 'Broad money (% of GDP)': 63.1321973026696, 'Total reserves (includes gold, current US$)': 164620171357.293, 'timestamp': 1635359367247}
{'year': '1991', 'country': 'US', 'Stocks traded, total value (% of GDP)': 34.7482522694799, 'Inflation, consumer prices (annual %)': 4.23496396453853, 'Stocks traded, turnover ratio of domestic shares (%)': 51.4435202733189, 'Tax revenue (% of GDP)': 10.3203424286825, 'GDP per capita (current US$)': 24342.2589048189, 'Central government debt, total (% of GDP)': 44.0615972806026, 'Gini index (World Bank estimate)': 38, 'Broad money (% of GDP)': 70.0032428680854, 'Total reserves (includes gold, current US$)': 159272850288.003, 'timestamp': 1635359356949}
{'year': '1992', 'country': 'US', 'Stocks traded, total value (% of GDP)': 38.

In [ ]:
import pandas as pd
df = pd.DataFrame(kafkaListDictionaries)
sdf = spark.createDataFrame(df)

In [ ]:
# Write the consumed messages to CSV's in the data lake
sdf.write.mode('overwrite').csv('/mnt/group1-capstone/consumed_financial_data_v2',header='true')

In [ ]:
from pyspark.sql.functions import *
SQLdf = spark.read.option("header","true").csv('/mnt/group1-capstone/consumed_financial_data_v2/*')

In [ ]:
# Select Database
database = "group1"
table = "dbo.capstone_sql_table" # need to decide how to do the table names, likely will need a different table for each dataframe
user = "group1user"
password  = "everythingIsAwesome!"
server = "database2108.database.windows.net"

In [ ]:
SQLdf.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()